In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

<h2 align='center'> Lithium </h2>

In [38]:
%matplotlib qt

lithium_data = pd.read_excel(r'C:\Users\glebf\OneDrive\Рабочий стол\Lithium_phase.xlsx')

lithium_temperature = np.array(lithium_data['T'])
lithium_pressure = np.array(lithium_data['P'])
lithium_density = np.array(lithium_data['D'])

lithium_temperature = np.log(lithium_temperature)
lithium_pressure = np.log(lithium_pressure)
lithium_density = np.log(lithium_density)


MIN_LITHIUM_TEMPERATURE = min(lithium_temperature)
MAX_LITHIUM_TEMPERATURE = max(lithium_temperature)
MIN_LITHIUM_PRESSURE = min(lithium_pressure)
MAX_LITHIUM_PRESSURE = max(lithium_pressure)
MIN_LITHIUM_DENSITY = min(lithium_density)
MAX_LITHIUM_DENSITY = max(lithium_density)

linearized_lithium_temperature = (lithium_temperature - MIN_LITHIUM_TEMPERATURE) / (MAX_LITHIUM_TEMPERATURE - MIN_LITHIUM_TEMPERATURE)
linearized_lithium_pressure = (lithium_pressure - MIN_LITHIUM_PRESSURE) / (MAX_LITHIUM_PRESSURE - MIN_LITHIUM_PRESSURE)
linearized_lithium_density = (lithium_density - MIN_LITHIUM_DENSITY) / (MAX_LITHIUM_DENSITY - MIN_LITHIUM_DENSITY)

In [3]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.title.set_text('Lithium')
ax.scatter(lithium_temperature, lithium_pressure, lithium_density)
ax.set_xlabel(r'$ln(T)$')
ax.set_ylabel(r'$ln(P)$')
ax.set_zlabel(r'$ln(\rho)$')
plt.grid(True)
plt.show()

In [4]:
inputs = tf.keras.layers.Input(shape=(2,), name='Lithium_input')
x = tf.keras.layers.Dense(10, activation='relu', name='Lithium_dense_1')(inputs)
# x = tf.keras.layers.Dense(10, activation='relu', name='Lithium_dense_2')(x)
outputs = tf.keras.layers.Dense(1, activation='linear', name='Lithium_output')(x)
lithium_model = tf.keras.models.Model(
    inputs=inputs,
    outputs=outputs,
    name='Lithium'
)
lithium_model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.mse
)
lithium_model.summary()

Model: "Lithium"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Lithium_input (InputLayer)  [(None, 2)]               0         
                                                                 
 Lithium_dense_1 (Dense)     (None, 10)                30        
                                                                 
 Lithium_output (Dense)      (None, 1)                 11        
                                                                 
Total params: 41 (164.00 Byte)
Trainable params: 41 (164.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [5]:
lithium_index = np.arange(len(linearized_lithium_temperature))
np.random.shuffle(lithium_index)

lithium_x_train = np.array([linearized_lithium_temperature[lithium_index], linearized_lithium_pressure[lithium_index]]).T
lithium_y_train = np.array([linearized_lithium_density[lithium_index]]).T
lithium_fit = lithium_model.fit(lithium_x_train, lithium_y_train, epochs=500, use_multiprocessing=True, validation_split=0.2)

Epoch 1/500
8/8 [==============================] - 3s 64ms/step - loss: 0.2569 - val_loss: 0.2330
Epoch 2/500
8/8 [==============================] - 0s 8ms/step - loss: 0.1864 - val_loss: 0.1671
Epoch 3/500
8/8 [==============================] - 0s 8ms/step - loss: 0.1310 - val_loss: 0.1159
Epoch 4/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0888 - val_loss: 0.0787
Epoch 5/500
8/8 [==============================] - 0s 6ms/step - loss: 0.0588 - val_loss: 0.0533
Epoch 6/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0394 - val_loss: 0.0369
Epoch 7/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0274 - val_loss: 0.0266
Epoch 8/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0202 - val_loss: 0.0205
Epoch 9/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0159 - val_loss: 0.0168
Epoch 10/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0134 - val_loss: 0.0142
Epoch 11/500
8/8 [==========

In [13]:
lithium_x_test = np.array([linearized_lithium_temperature[lithium_index], linearized_lithium_pressure[lithium_index]]).T
lithium_y_test = lithium_model.predict(lithium_x_test).T

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.set_title('Lithium')
ax.scatter(linearized_lithium_temperature[lithium_index], linearized_lithium_pressure[lithium_index], lithium_y_test, color='g', alpha=0.5)
ax.scatter(linearized_lithium_temperature[lithium_index], linearized_lithium_pressure[lithium_index], linearized_lithium_density[lithium_index], color='b', alpha=0.5)
ax.set_xlabel(r'$T$')
ax.set_ylabel(r'$P$')
ax.set_zlabel(r'$\rho$')
plt.legend(['Network data', 'Experimental data'])
ax.grid(True)
plt.show()

loss = lithium_fit.history['loss']
val_loss = lithium_fit.history['val_loss']

plt.figure()
plt.plot(range(len(loss)), loss)
plt.plot(range(len(val_loss)), val_loss)
plt.legend(['loss', 'val_loss'])
plt.xlabel('Epoch')
plt.ylabel('MSE')
plt.xlim([400, 500])
plt.ylim([0, 1e-5])
plt.grid(True)
plt.show()

plt.figure()
plt.scatter(linearized_lithium_density[lithium_index], lithium_y_test, alpha=.5)
plt.plot([0, 1], [0, 1])
# plt.xlim([0, 2e-1])
# plt.ylim([0, 2e-1])
plt.xlabel(r'$\rho_{exp}$')
plt.ylabel(r'$\rho_{network}$')
plt.grid(True)
plt.show()

10/10 [==============================] - 0s 2ms/step


<h2 align='center'> Natrium </h2>

In [39]:
natrium_data = pd.read_excel(r'C:\Users\glebf\OneDrive\Рабочий стол\Natrium_phase.xlsx')
natrium_temperature = np.array(natrium_data['T'])
natrium_pressure = np.array(natrium_data['P'])
natrium_density = np.array(natrium_data['D'])

natrium_temperature = np.log(natrium_temperature)
natrium_pressure = np.log(natrium_pressure)
natrium_density = np.log(natrium_density)


MIN_NATRIUM_TEMPERATURE = min(natrium_temperature)
MAX_NATRIUM_TEMPERATURE = max(natrium_temperature)
MIN_NATRIUM_PRESSURE = min(natrium_pressure)
MAX_NATRIUM_PRESSURE = max(natrium_pressure)
MIN_NATRIUM_DENSITY = min(natrium_density)
MAX_NATRIUM_DENSITY = max(natrium_density)

linearized_natrium_temperature = (natrium_temperature - MIN_NATRIUM_TEMPERATURE) / (MAX_NATRIUM_TEMPERATURE - MIN_NATRIUM_TEMPERATURE)
linearized_natrium_pressure = (natrium_pressure - MIN_NATRIUM_PRESSURE) / (MAX_NATRIUM_PRESSURE - MIN_NATRIUM_PRESSURE)
linearized_natrium_density = (natrium_density - MIN_NATRIUM_DENSITY) / (MAX_NATRIUM_DENSITY - MIN_NATRIUM_DENSITY)

In [25]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.title.set_text('Natrium')
ax.scatter(natrium_temperature, natrium_pressure, natrium_density)
ax.set_xlabel(r'$T$')
ax.set_ylabel(r'$P$')
ax.set_zlabel(r'$\rho$')
plt.grid(True)
plt.show()

In [41]:
inputs = tf.keras.layers.Input(shape=(2,), name='Natrium_input')
x = tf.keras.layers.Dense(20, activation='relu', name='Natrium_dense_1')(inputs)
# x = tf.keras.layers.Dense(20, activation='gelu', name='Natrium_dense_2')(x)
outputs = tf.keras.layers.Dense(1, activation='linear', name='Natrium_output')(x)
natrium_model = tf.keras.models.Model(
    inputs=inputs,
    outputs=outputs,
    name='Natrium'
)
natrium_model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.mse
)
natrium_model.summary()

Model: "Natrium"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Natrium_input (InputLayer)  [(None, 2)]               0         
                                                                 
 Natrium_dense_1 (Dense)     (None, 20)                60        
                                                                 
 Natrium_output (Dense)      (None, 1)                 21        
                                                                 
Total params: 81 (324.00 Byte)
Trainable params: 81 (324.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [42]:
natrium_index = np.arange(len(linearized_natrium_temperature))
np.random.shuffle(natrium_index)

natrium_x_train = np.array([linearized_natrium_temperature[natrium_index], linearized_natrium_pressure[natrium_index]]).T
natrium_y_train = np.array([linearized_natrium_density[natrium_index]]).T
natrium_fit = natrium_model.fit(natrium_x_train, natrium_y_train, epochs=500, use_multiprocessing=True, validation_split=0.2)

Epoch 1/500
10/10 [==============================] - 2s 19ms/step - loss: 0.1307 - val_loss: 0.1133
Epoch 2/500
10/10 [==============================] - 0s 6ms/step - loss: 0.0827 - val_loss: 0.0690
Epoch 3/500
10/10 [==============================] - 0s 6ms/step - loss: 0.0501 - val_loss: 0.0421
Epoch 4/500
10/10 [==============================] - 0s 5ms/step - loss: 0.0330 - val_loss: 0.0277
Epoch 5/500
10/10 [==============================] - 0s 6ms/step - loss: 0.0252 - val_loss: 0.0214
Epoch 6/500
10/10 [==============================] - 0s 5ms/step - loss: 0.0219 - val_loss: 0.0182
Epoch 7/500
10/10 [==============================] - 0s 5ms/step - loss: 0.0194 - val_loss: 0.0160
Epoch 8/500
10/10 [==============================] - 0s 5ms/step - loss: 0.0170 - val_loss: 0.0140
Epoch 9/500
10/10 [==============================] - 0s 5ms/step - loss: 0.0148 - val_loss: 0.0123
Epoch 10/500
10/10 [==============================] - 0s 5ms/step - loss: 0.0129 - val_loss: 0.0107
Epoch 11

In [48]:
natirum_x_test = np.array([linearized_natrium_temperature[natrium_index], linearized_natrium_pressure[natrium_index]]).T
natrium_y_test = natrium_model.predict(natirum_x_test).T

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.set_title('Natrium')
ax.scatter(linearized_natrium_temperature[natrium_index], linearized_natrium_pressure[natrium_index], natrium_y_test, color='g', alpha=0.5)
ax.scatter(linearized_natrium_temperature[natrium_index], linearized_natrium_pressure[natrium_index], linearized_natrium_density[natrium_index], color='b', alpha=0.5)
ax.set_xlabel(r'$T$')
ax.set_ylabel(r'$P$')
ax.set_zlabel(r'$\rho$')
ax.grid(True)
plt.legend(['Network data', 'Experimental data'])
plt.show()

loss = natrium_fit.history['loss']
val_loss = natrium_fit.history['val_loss']

plt.figure()
plt.plot(range(len(loss)), loss)
plt.plot(range(len(val_loss)), val_loss)
plt.legend(['loss', 'val_loss'])
plt.xlabel('Epoch')
plt.ylabel('MSE')
plt.xlim([400, 500])
plt.ylim([0, 3e-6])
plt.grid(True)
plt.show()

plt.figure()
plt.scatter(linearized_natrium_density[natrium_index], natrium_y_test, alpha=.5)
plt.plot([0, 1], [0, 1])
# plt.xlim([0, 2e-1])
# plt.ylim([0, 2e-1])
plt.xlabel(r'$\rho_{exp}$')
plt.ylabel(r'$\rho_{network}$')
plt.grid(True)
plt.show()

12/12 [==============================] - 0s 2ms/step


<h2 align='center'> Kalium </h2>

In [40]:
kalium_data = pd.read_excel(r'C:\Users\glebf\OneDrive\Рабочий стол\Kalium_phase.xlsx')
kalium_temperature = np.array(kalium_data['T'])
kalium_pressure = np.array(kalium_data['P'])
kalium_density = np.array(kalium_data['D'])

kalium_temperature = np.log(kalium_temperature)
kalium_pressure = np.log(kalium_pressure)
kalium_density = np.log(kalium_density)

MIN_KALIUM_TEMPERATURE = min(kalium_temperature)
MAX_KALIUM_TEMPERATURE = max(kalium_temperature)
MIN_KALIUM_PRESSURE = min(kalium_pressure)
MAX_KALIUM_PRESSURE = max(kalium_pressure)
MIN_KALIUM_DENSITY = min(kalium_density)
MAX_KALIUM_DENSITY = max(kalium_density)

linearized_kalium_temperature = (kalium_temperature - MIN_KALIUM_TEMPERATURE) / (MAX_KALIUM_TEMPERATURE - MIN_KALIUM_TEMPERATURE)
linearized_kalium_pressure = (kalium_pressure - MIN_KALIUM_PRESSURE) / (MAX_KALIUM_PRESSURE - MIN_KALIUM_PRESSURE)
linearized_kalium_density = (kalium_density - MIN_KALIUM_DENSITY) / (MAX_KALIUM_DENSITY - MIN_KALIUM_DENSITY)

In [53]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.title.set_text('Kalium')
ax.scatter(kalium_temperature, kalium_pressure, kalium_density)
ax.set_xlabel(r'$T$')
ax.set_ylabel(r'$P$')
ax.set_zlabel(r'$\rho$')
plt.grid(True)
plt.show()

In [77]:
inputs = tf.keras.layers.Input(shape=(2,), name='Kalium_input')
x = tf.keras.layers.Dense(10, activation='relu', name='Kalium_dense_1')(inputs)
# x = tf.keras.layers.Dense(10, activation='relu', name='Kalium_dense_2')(x)
outputs = tf.keras.layers.Dense(1, activation='linear', name='Kalium_output')(x)
kalium_model = tf.keras.models.Model(
    inputs=inputs,
    outputs=outputs,
    name='Kalium'
)
kalium_model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.mse
)
kalium_model.summary()

Model: "Kalium"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Kalium_input (InputLayer)   [(None, 2)]               0         
                                                                 
 Kalium_dense_1 (Dense)      (None, 10)                30        
                                                                 
 Kalium_output (Dense)       (None, 1)                 11        
                                                                 
Total params: 41 (164.00 Byte)
Trainable params: 41 (164.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [78]:
kalium_index = np.arange(len(linearized_kalium_temperature))
np.random.shuffle(kalium_index)

kalium_x_train = np.array([linearized_kalium_temperature[kalium_index], linearized_kalium_pressure[kalium_index]]).T
kalium_y_train = np.array([linearized_kalium_density[kalium_index]]).T
kalium_fit = kalium_model.fit(kalium_x_train, kalium_y_train, epochs=600, use_multiprocessing=True, validation_split=0.2)

Epoch 1/600
10/10 [==============================] - 3s 32ms/step - loss: 0.1147 - val_loss: 0.0883
Epoch 2/600
10/10 [==============================] - 0s 9ms/step - loss: 0.0728 - val_loss: 0.0565
Epoch 3/600
10/10 [==============================] - 0s 9ms/step - loss: 0.0483 - val_loss: 0.0397
Epoch 4/600
10/10 [==============================] - 0s 8ms/step - loss: 0.0356 - val_loss: 0.0333
Epoch 5/600
10/10 [==============================] - 0s 8ms/step - loss: 0.0301 - val_loss: 0.0310
Epoch 6/600
10/10 [==============================] - 0s 8ms/step - loss: 0.0281 - val_loss: 0.0294
Epoch 7/600
10/10 [==============================] - 0s 7ms/step - loss: 0.0262 - val_loss: 0.0273
Epoch 8/600
10/10 [==============================] - 0s 8ms/step - loss: 0.0242 - val_loss: 0.0250
Epoch 9/600
10/10 [==============================] - 0s 8ms/step - loss: 0.0223 - val_loss: 0.0230
Epoch 10/600
10/10 [==============================] - 0s 8ms/step - loss: 0.0205 - val_loss: 0.0211
Epoch 11

In [85]:
kalium_x_test = np.array([linearized_kalium_temperature[kalium_index], linearized_kalium_pressure[kalium_index]]).T
kalium_y_test = natrium_model.predict(kalium_x_test).T

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.set_title('Kalium')
ax.scatter(linearized_kalium_temperature[kalium_index], linearized_kalium_pressure[kalium_index], kalium_y_test, color='g', alpha=0.5)
ax.scatter(linearized_kalium_temperature[kalium_index], linearized_kalium_pressure[kalium_index], linearized_kalium_density[kalium_index], color='b', alpha=0.5)
ax.set_xlabel(r'$T$')
ax.set_ylabel(r'$P$')
ax.set_zlabel(r'$\rho$')
ax.grid(True)
plt.legend(['Network data', 'Experimental data'])
plt.show()

loss = kalium_fit.history['loss']
val_loss = kalium_fit.history['val_loss']

plt.figure()
plt.plot(range(len(loss)), loss)
plt.plot(range(len(val_loss)), val_loss)
plt.legend(['loss', 'val_loss'])
plt.xlabel('Epoch')
plt.ylabel('MSE')
plt.xlim([400, 500])
plt.ylim([0, 2e-5])
plt.grid(True)
plt.show()

plt.figure()
plt.scatter(linearized_kalium_density[kalium_index], kalium_y_test, alpha=.5)
plt.plot([0, 1], [0, 1])
# plt.xlim([0, 2e-1])
# plt.ylim([0, 2e-1])
plt.xlabel(r'$\rho_{exp}$')
plt.ylabel(r'$\rho_{network}$')
plt.grid(True)
plt.show()

13/13 [==============================] - 0s 3ms/step


<h2 align='center'> Rubidium </h2>

In [41]:
rubidium_data = pd.read_excel(r'C:\Users\glebf\OneDrive\Рабочий стол\Rubidium_phase.xlsx')
rubidium_temperature = np.array(rubidium_data['T'])
rubidium_pressure = np.array(rubidium_data['P'])
rubidium_density = np.array(rubidium_data['D'])

rubidium_temperature = np.log(rubidium_temperature)
rubidium_pressure = np.log(rubidium_pressure)
rubidium_density = np.log(rubidium_density)

MIN_RUBIDIUM_TEMPERATURE = min(rubidium_temperature)
MAX_RUBIDIUM_TEMPERATURE = max(rubidium_temperature)
MIN_RUBIDIUM_PRESSURE = min(rubidium_pressure)
MAX_RUBIDIUM_PRESSURE = max(rubidium_pressure)
MIN_RUBIDIUM_DENSITY = min(rubidium_density)
MAX_RUBIDIUM_DENSITY = max(rubidium_density)

linearized_rubidium_temperature = (rubidium_temperature - MIN_RUBIDIUM_TEMPERATURE) / (MAX_RUBIDIUM_TEMPERATURE - MIN_RUBIDIUM_TEMPERATURE)
linearized_rubidium_pressure = (rubidium_pressure - MIN_RUBIDIUM_PRESSURE) / (MAX_RUBIDIUM_PRESSURE - MIN_RUBIDIUM_PRESSURE)
linearized_rubidium_density = (rubidium_density - MIN_RUBIDIUM_DENSITY) / (MAX_RUBIDIUM_DENSITY - MIN_RUBIDIUM_DENSITY)

In [87]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.title.set_text('Rubidium')
ax.scatter(rubidium_temperature, rubidium_pressure, rubidium_density)
ax.set_xlabel(r'$T$')
ax.set_ylabel(r'$P$')
ax.set_zlabel(r'$\rho$')
plt.grid(True)
plt.show()

In [88]:
inputs = tf.keras.layers.Input(shape=(2,), name='Rubidium_input')
x = tf.keras.layers.Dense(20, activation='relu', name='Rubidium_dense_1')(inputs)
# x = tf.keras.layers.Dense(20, activation='gelu', name='Rubidium_dense_2')(x)
outputs = tf.keras.layers.Dense(1, activation='linear', name='Rubidium_output')(x)
rubidium_model = tf.keras.models.Model(
    inputs=inputs,
    outputs=outputs,
    name='Rubidium'
)
rubidium_model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.mse
)
rubidium_model.summary()

Model: "Rubidium"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Rubidium_input (InputLayer  [(None, 2)]               0         
 )                                                               
                                                                 
 Rubidium_dense_1 (Dense)    (None, 20)                60        
                                                                 
 Rubidium_output (Dense)     (None, 1)                 21        
                                                                 
Total params: 81 (324.00 Byte)
Trainable params: 81 (324.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [89]:
rubidium_index = np.arange(len(linearized_rubidium_temperature))
np.random.shuffle(rubidium_index)

rubidium_x_train = np.array([linearized_rubidium_temperature[rubidium_index], linearized_rubidium_pressure[rubidium_index]]).T
rubidium_y_train = np.array([linearized_rubidium_density[rubidium_index]]).T
rubidium_fit = rubidium_model.fit(rubidium_x_train, rubidium_y_train, epochs=500, use_multiprocessing=True, validation_split=0.2)

Epoch 1/500
10/10 [==============================] - 4s 76ms/step - loss: 0.4133 - val_loss: 0.3509
Epoch 2/500
10/10 [==============================] - 0s 12ms/step - loss: 0.3173 - val_loss: 0.2646
Epoch 3/500
10/10 [==============================] - 0s 10ms/step - loss: 0.2365 - val_loss: 0.1905
Epoch 4/500
10/10 [==============================] - 0s 13ms/step - loss: 0.1673 - val_loss: 0.1308
Epoch 5/500
10/10 [==============================] - 0s 12ms/step - loss: 0.1129 - val_loss: 0.0858
Epoch 6/500
10/10 [==============================] - 0s 9ms/step - loss: 0.0744 - val_loss: 0.0533
Epoch 7/500
10/10 [==============================] - 0s 8ms/step - loss: 0.0469 - val_loss: 0.0322
Epoch 8/500
10/10 [==============================] - 0s 11ms/step - loss: 0.0290 - val_loss: 0.0197
Epoch 9/500
10/10 [==============================] - 0s 11ms/step - loss: 0.0191 - val_loss: 0.0137
Epoch 10/500
10/10 [==============================] - 0s 12ms/step - loss: 0.0152 - val_loss: 0.0121
E

In [91]:
rubidium_x_test = np.array([linearized_rubidium_temperature[rubidium_index], linearized_rubidium_pressure[rubidium_index]]).T
rubidium_y_test = rubidium_model.predict(rubidium_x_test).T

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.set_title('Rubidium')
ax.scatter(linearized_rubidium_temperature[rubidium_index], linearized_rubidium_pressure[rubidium_index], rubidium_y_test, color='g', alpha=0.5)
ax.scatter(linearized_rubidium_temperature[rubidium_index], linearized_rubidium_pressure[rubidium_index], linearized_rubidium_density[rubidium_index], color='b',alpha=.5)
ax.set_xlabel(r'$T$')
ax.set_ylabel(r'$P$')
ax.set_zlabel(r'$\rho$')
ax.grid(True)
plt.legend(['Network data', 'Experimental data'])
plt.show()

loss = rubidium_fit.history['loss']
val_loss = rubidium_fit.history['val_loss']

plt.figure()
plt.plot(range(len(loss)), loss)
plt.plot(range(len(val_loss)), val_loss)
plt.legend(['loss', 'val_loss'])
plt.xlabel('Epoch')
plt.ylabel('MSE')
plt.xlim([400, 500])
plt.ylim([0, 2e-5])
plt.grid(True)
plt.show()

plt.figure()
plt.scatter(linearized_rubidium_density[rubidium_index], rubidium_y_test, alpha=.5)
plt.plot([0, 1], [0, 1])
# plt.xlim([0, 2e-1])
# plt.ylim([0, 2e-1])
plt.xlabel(r'$\rho_{exp}$')
plt.ylabel(r'$\rho_{network}$')
plt.grid(True)
plt.show()

13/13 [==============================] - 0s 2ms/step


<h2 align='center'> Cesium </h2>

In [42]:
cesium_data = pd.read_excel(r'C:\Users\glebf\OneDrive\Рабочий стол\Cesium_phase.xlsx')
cesium_temperature = np.array(cesium_data['T'])
cesium_pressure = np.array(cesium_data['P'])
cesium_density = np.array(cesium_data['D'])

cesium_temperature = np.log(cesium_temperature)
cesium_pressure = np.log(cesium_pressure)
cesium_density = np.log(cesium_density)

MIN_CESIUM_TEMPERATURE = min(cesium_temperature)
MAX_CESIUM_TEMPERATURE = max(cesium_temperature)
MIN_CESIUM_PRESSURE = min(cesium_pressure)
MAX_CESIUM_PRESSURE = max(cesium_pressure)
MIN_CESIUM_DENSITY = min(cesium_density)
MAX_CESIUM_DENSITY = max(cesium_density)

linearized_cesium_temperature = (cesium_temperature - MIN_CESIUM_TEMPERATURE) / (MAX_CESIUM_TEMPERATURE - MIN_CESIUM_TEMPERATURE)
linearized_cesium_pressure = (cesium_pressure - MIN_CESIUM_PRESSURE) / (MAX_CESIUM_PRESSURE - MIN_CESIUM_PRESSURE)
linearized_cesium_density = (cesium_density - MIN_CESIUM_DENSITY) / (MAX_CESIUM_DENSITY - MIN_CESIUM_DENSITY)

In [96]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.title.set_text('Cesium')
ax.scatter(cesium_temperature, cesium_pressure, cesium_density)
ax.set_xlabel(r'$T$')
ax.set_ylabel(r'$P$')
ax.set_zlabel(r'$\rho$')
plt.grid(True)
plt.show()

In [97]:
inputs = tf.keras.layers.Input(shape=(2,), name='Cesium_input')
x = tf.keras.layers.Dense(20, activation='relu', name='Cesium_dense_2')(inputs)
# x = tf.keras.layers.Dense(20, activation='gelu', name='Cesium_dense_3')(x)
outputs = tf.keras.layers.Dense(1, activation='linear', name='Cesium_output')(x)
cesium_model = tf.keras.models.Model(
    inputs=inputs,
    outputs=outputs,
    name='Cesium'
)
cesium_model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.mse
)
cesium_model.summary()

Model: "Cesium"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Cesium_input (InputLayer)   [(None, 2)]               0         
                                                                 
 Cesium_dense_2 (Dense)      (None, 20)                60        
                                                                 
 Cesium_output (Dense)       (None, 1)                 21        
                                                                 
Total params: 81 (324.00 Byte)
Trainable params: 81 (324.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [98]:
cesium_index = np.arange(len(linearized_cesium_temperature))
np.random.shuffle(cesium_index)

cesium_x_train = np.array([linearized_cesium_temperature[cesium_index], linearized_cesium_pressure[cesium_index]]).T
cesium_y_train = np.array([linearized_cesium_density[cesium_index]]).T
cesium_fit = cesium_model.fit(cesium_x_train, cesium_y_train, epochs=500, use_multiprocessing=True, validation_split=0.2)

Epoch 1/500
10/10 [==============================] - 3s 30ms/step - loss: 0.5996 - val_loss: 0.5725
Epoch 2/500
10/10 [==============================] - 0s 8ms/step - loss: 0.4424 - val_loss: 0.4185
Epoch 3/500
10/10 [==============================] - 0s 9ms/step - loss: 0.3165 - val_loss: 0.2966
Epoch 4/500
10/10 [==============================] - 0s 8ms/step - loss: 0.2186 - val_loss: 0.2033
Epoch 5/500
10/10 [==============================] - 0s 8ms/step - loss: 0.1458 - val_loss: 0.1343
Epoch 6/500
10/10 [==============================] - 0s 7ms/step - loss: 0.0940 - val_loss: 0.0858
Epoch 7/500
10/10 [==============================] - 0s 9ms/step - loss: 0.0579 - val_loss: 0.0539
Epoch 8/500
10/10 [==============================] - 0s 8ms/step - loss: 0.0354 - val_loss: 0.0335
Epoch 9/500
10/10 [==============================] - 0s 9ms/step - loss: 0.0219 - val_loss: 0.0200
Epoch 10/500
10/10 [==============================] - 0s 9ms/step - loss: 0.0130 - val_loss: 0.0117
Epoch 11

In [101]:
cesium_x_test = np.array([linearized_cesium_temperature[cesium_index], linearized_cesium_pressure[cesium_index]]).T
cesium_y_test = cesium_model.predict(cesium_x_test).T

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.set_title('Cesium')
ax.scatter(linearized_cesium_temperature[cesium_index], linearized_cesium_pressure[cesium_index], cesium_y_test, color='g', alpha=0.5)
ax.scatter(linearized_cesium_temperature[cesium_index], linearized_cesium_pressure[cesium_index], linearized_cesium_density[cesium_index], color='b',alpha=.5)
ax.set_xlabel(r'$T$')
ax.set_ylabel(r'$P$')
ax.set_zlabel(r'$\rho$')
ax.grid(True)
plt.legend(['Network data', 'Experimental data'])
plt.show()

loss = cesium_fit.history['loss']
val_loss = cesium_fit.history['val_loss']

plt.figure()
plt.plot(range(len(loss)), loss)
plt.plot(range(len(val_loss)), val_loss)
plt.legend(['loss', 'val_loss'])
plt.xlabel('Epoch')
plt.ylabel('MSE')
plt.xlim([400, 500])
plt.ylim([0, 2e-5])
plt.grid(True)
plt.show()

plt.figure()
plt.scatter(linearized_cesium_density[cesium_index], cesium_y_test, alpha=.5)
plt.plot([0, 1], [0, 1])
# plt.xlim([0, 2e-1])
# plt.ylim([0, 2e-1])
plt.xlabel(r'$\rho_{exp}$')
plt.ylabel(r'$\rho_{network}$')
plt.grid(True)
plt.show()

13/13 [==============================] - 0s 3ms/step


<h2 align='center'> Mutual network </h2>

In [43]:
MAX_ALKALINE_DENSITY = max([
    MAX_CESIUM_DENSITY, 
    MAX_KALIUM_DENSITY, 
    MAX_LITHIUM_DENSITY,
    MAX_NATRIUM_DENSITY,
    MAX_RUBIDIUM_DENSITY
])

MIN_ALKALINE_DENSITY = min([
    MIN_CESIUM_DENSITY, 
    MIN_KALIUM_DENSITY, 
    MIN_LITHIUM_DENSITY,
    MIN_NATRIUM_DENSITY,
    MIN_RUBIDIUM_DENSITY
])

MAX_ALKALINE_PRESSURE = max([
    MAX_CESIUM_PRESSURE, 
    MAX_KALIUM_PRESSURE, 
    MAX_LITHIUM_PRESSURE,
    MAX_NATRIUM_PRESSURE,
    MAX_RUBIDIUM_PRESSURE
])

MIN_ALKALINE_PRESSURE = min([
    MIN_CESIUM_PRESSURE, 
    MIN_KALIUM_PRESSURE, 
    MIN_LITHIUM_PRESSURE,
    MIN_NATRIUM_PRESSURE,
    MIN_RUBIDIUM_PRESSURE
])

MAX_ALKALINE_TEMPERATURE = max([
    MAX_CESIUM_TEMPERATURE, 
    MAX_KALIUM_TEMPERATURE, 
    MAX_LITHIUM_TEMPERATURE,
    MAX_NATRIUM_TEMPERATURE,
    MAX_RUBIDIUM_TEMPERATURE
])

MIN_ALKALINE_TEMPERATURE = min([
    MIN_CESIUM_TEMPERATURE, 
    MIN_KALIUM_TEMPERATURE, 
    MIN_LITHIUM_TEMPERATURE,
    MIN_NATRIUM_TEMPERATURE,
    MIN_RUBIDIUM_TEMPERATURE
])

def minmax(x, maxx, minx):
    return (x - minx) / (maxx - minx)

data = [
    {
        'M': [6.941e-3] * len(linearized_lithium_temperature),
        'temperature': minmax(lithium_temperature, MAX_ALKALINE_TEMPERATURE, MIN_ALKALINE_TEMPERATURE),
        'pressure': minmax(lithium_pressure, MAX_ALKALINE_PRESSURE, MIN_ALKALINE_PRESSURE),
        'density': minmax(lithium_density, MAX_ALKALINE_DENSITY, MIN_ALKALINE_DENSITY)
    },
    {
        'M': [22.989e-3] * len(linearized_natrium_temperature),
        'temperature': minmax(natrium_temperature, MAX_ALKALINE_TEMPERATURE, MIN_ALKALINE_TEMPERATURE),
        'pressure': minmax(natrium_pressure, MAX_ALKALINE_PRESSURE, MIN_ALKALINE_PRESSURE),
        'density': minmax(natrium_density, MAX_ALKALINE_DENSITY, MIN_ALKALINE_DENSITY)
    },
    {
        'M': [39.098e-3] * len(linearized_kalium_temperature),
        'temperature': minmax(kalium_temperature, MAX_ALKALINE_TEMPERATURE, MIN_ALKALINE_TEMPERATURE),
        'pressure': minmax(kalium_pressure, MAX_ALKALINE_PRESSURE, MIN_ALKALINE_PRESSURE),
        'density': minmax(kalium_density, MAX_ALKALINE_DENSITY, MIN_ALKALINE_DENSITY)
    },
    {
        'M': [85.468e-3] * len(linearized_rubidium_temperature),
        'temperature': minmax(rubidium_temperature, MAX_ALKALINE_TEMPERATURE, MIN_ALKALINE_TEMPERATURE),
        'pressure': minmax(rubidium_pressure, MAX_ALKALINE_PRESSURE, MIN_ALKALINE_PRESSURE),
        'density': minmax(rubidium_density, MAX_ALKALINE_DENSITY, MIN_ALKALINE_DENSITY)
    },
    {
        'M': [132.905e-3] * len(linearized_cesium_temperature),
        'temperature': minmax(cesium_temperature, MAX_ALKALINE_TEMPERATURE, MIN_ALKALINE_TEMPERATURE),
        'pressure': minmax(cesium_pressure, MAX_ALKALINE_PRESSURE, MIN_ALKALINE_PRESSURE),
        'density': minmax(cesium_density, MAX_ALKALINE_DENSITY, MIN_ALKALINE_DENSITY)
    }
]

alkaline_temperature = []
alkaline_temperature.extend(data[0]['temperature'])
alkaline_temperature.extend(data[1]['temperature'])
alkaline_temperature.extend(data[2]['temperature'])
alkaline_temperature.extend(data[3]['temperature'])
alkaline_temperature.extend(data[4]['temperature'])
alkaline_temperature = np.array(alkaline_temperature, dtype=np.float64)

alkaline_pressure = []
alkaline_pressure.extend(data[0]['pressure'])
alkaline_pressure.extend(data[1]['pressure'])
alkaline_pressure.extend(data[2]['pressure'])
alkaline_pressure.extend(data[3]['pressure'])
alkaline_pressure.extend(data[4]['pressure'])
alkaline_pressure = np.array(alkaline_pressure, dtype=np.float64)

alkaline_density = []
alkaline_density.extend(data[0]['density'])
alkaline_density.extend(data[1]['density'])
alkaline_density.extend(data[2]['density'])
alkaline_density.extend(data[3]['density'])
alkaline_density.extend(data[4]['density'])
alkaline_density = np.array(alkaline_density, dtype=np.float64)

alkaline_M = []
alkaline_M.extend(data[0]['M'])
alkaline_M.extend(data[1]['M'])
alkaline_M.extend(data[2]['M'])
alkaline_M.extend(data[3]['M'])
alkaline_M.extend(data[4]['M'])
alkaline_M = np.array(alkaline_M, dtype=np.float64)

In [29]:
inputs = tf.keras.layers.Input(shape=(3,), name='Alkaline_input')
x = tf.keras.layers.Dense(20, activation='relu', name='Alkaline_dense_1')(inputs)
x = tf.keras.layers.Dense(20, activation='relu', name='Alkaline_dense_2')(x)
outputs = tf.keras.layers.Dense(1, activation='linear', name='Alkaline_output')(x)
alkaline_model = tf.keras.models.Model(
    inputs=inputs,
    outputs=outputs,
    name='Alkaline'
)
alkaline_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=tf.keras.losses.mse
)
alkaline_model.summary()

Model: "Alkaline"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Alkaline_input (InputLayer  [(None, 3)]               0         
 )                                                               
                                                                 
 Alkaline_dense_1 (Dense)    (None, 20)                80        
                                                                 
 Alkaline_dense_2 (Dense)    (None, 20)                420       
                                                                 
 Alkaline_output (Dense)     (None, 1)                 21        
                                                                 
Total params: 521 (2.04 KB)
Trainable params: 521 (2.04 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [30]:
# index = np.arange(len(alkaline_temperature))

index = (alkaline_density < 1) & (alkaline_density > 0)
# np.random.shuffle(index)

alkaline_x_train = np.array([alkaline_M[index], alkaline_temperature[index], alkaline_pressure[index]]).T
alkaline_y_train = np.array([alkaline_density[index]]).T
alkaline_fit = alkaline_model.fit(alkaline_x_train, alkaline_y_train, epochs=100, use_multiprocessing=True, validation_split=0.2)

Epoch 1/100
47/47 [==============================] - 3s 6ms/step - loss: 0.3440 - val_loss: 0.0665
Epoch 2/100
47/47 [==============================] - 0s 3ms/step - loss: 0.0124 - val_loss: 0.0111
Epoch 3/100
47/47 [==============================] - 0s 3ms/step - loss: 0.0066 - val_loss: 0.0074
Epoch 4/100
47/47 [==============================] - 0s 3ms/step - loss: 0.0046 - val_loss: 0.0072
Epoch 5/100
47/47 [==============================] - 0s 3ms/step - loss: 0.0038 - val_loss: 0.0065
Epoch 6/100
47/47 [==============================] - 0s 3ms/step - loss: 0.0032 - val_loss: 0.0046
Epoch 7/100
47/47 [==============================] - 0s 3ms/step - loss: 0.0028 - val_loss: 0.0050
Epoch 8/100
47/47 [==============================] - 0s 3ms/step - loss: 0.0025 - val_loss: 0.0032
Epoch 9/100
47/47 [==============================] - 0s 3ms/step - loss: 0.0023 - val_loss: 0.0034
Epoch 10/100
47/47 [==============================] - 0s 3ms/step - loss: 0.0020 - val_loss: 0.0023
Epoch 11/

In [47]:
x_test = np.array([data[0]['M'], data[0]['temperature'], data[0]['pressure']]).T
y_test = alkaline_model.predict(x_test).T

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.set_title('Lithium')
ax.scatter(data[0]['temperature'], data[0]['pressure'], y_test, color='g', alpha=0.5)
ax.scatter(data[0]['temperature'], data[0]['pressure'], data[0]['density'], color='b',alpha=.5)
ax.set_xlabel(r'$T$')
ax.set_ylabel(r'$P$')
ax.set_zlabel(r'$\rho$')
plt.legend(['Network data', 'Experimental data'])
ax.grid(True)
plt.show()

10/10 [==============================] - 0s 2ms/step


In [37]:
alkaline_x_test = np.array([alkaline_M[index], alkaline_temperature[index], alkaline_pressure[index]]).T
alkaline_y_test = alkaline_model.predict(alkaline_x_test).T

plt.figure()
plt.scatter(alkaline_density[index], alkaline_y_test, alpha=0.1)
plt.plot([0, 1], [0, 1])
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.xlabel(r'$\rho_{exp}$')
plt.ylabel(r'$\rho_{network}$')
plt.grid(True)
plt.show()

58/58 [==============================] - 0s 1ms/step


In [36]:
loss = alkaline_fit.history['loss']
val_loss = alkaline_fit.history['val_loss']

plt.figure()
plt.plot(range(len(loss)), loss)
plt.plot(range(len(val_loss)), val_loss)
plt.legend(['loss', 'val_loss'])
plt.xlabel('Epoch')
plt.ylabel('MSE')
plt.xlim([50, 100])
plt.ylim([0, 1e-3])
plt.grid(True)
plt.show()